# Генерация данных

Данный журнал проводит генерацию данных по принципу свёртки чётких изображений органических структур с изображениями размытых и чётких сфер.
На вход в блоке входных данных (констант) указываются:
- *PATHS* - лист с путями до .tif/.tiff изображений с чёткими органическими изображениями;
- *SPHERES_PATH* - Путь до снимков сфер;
- *BEAD_SHAPE* - Размер изображения чёткой сферы (число слоев, ширина, высота);
- *Z_SCALE, X_SCALE, Y_SCALE* - Масштабы съемки сфер вдоль осей OZ, OX, OY соответственно (в мкм/пксл);
- *BEAD_DIAMETER* - Диаметр сферы (в мкм/пксл);
- *IMG_SPLIT_SHAPE* - Размер нарезки входных чётких изображений органических объектов (число слоев, ширина, высота);
- *DATA_SHAPE* - Размер изображений в итоговом датасете. Размеры должны быть НЕ МЕНЬШЕ чем в *IMG_SPLIT_SHAPE* вдоль каждой из осей (число слоев, ширина, высота);
- *DATASET_NAME, DATASET_PATH* - Путь и название датасета;
- *DESCRIPTION* - Описание датасета (необязательно).

## Блок входных данных

In [1]:
# images paths
#PATHS = [#"./data/synthetic_data/synthetic_spheres.tiff",
#         #"./data/synthetic_data/synthetic_sticks.tiff",
#         "./data/raw_neurons/good_Project_Series005_z00_denoised.tif",
#         "./data/raw_neurons/good_Project_Series008_z00.tif",
#         "./data/raw_neurons/bad_Project_Series018_z00_denoised.tif",
#         "./data/raw_neurons/bad_Project_Series016.tif",
    
#         "./data/raw_neurons/moded_bad_Project_Series016.tif",
#         "./data/raw_neurons/moded_bad_Project_Series018_z00_denoised.tif",
#         "./data/raw_neurons/moded_good_Project_Series005_z00_denoised.tif",
#         "./data/raw_neurons/moded_good_Project_Series008_z00.tif",
    
#         "./data/raw_neurons/21.tif",
#         "./data/raw_neurons/24.tif",
#         "./data/raw_neurons/30.tiff"
#        ]  

PATHS = [
    "./data/raw_neurons/norm_010914_2.tif",
    "./data/raw_neurons/norm_010914.tif",
    "./data/raw_neurons/norm_042114_2.tif",
    "./data/raw_neurons/norm_042114_3.tif",
    "./data/raw_neurons/norm_042114.tif",
    "./data/raw_neurons/norm_smallim2.tif",
    "./data/raw_neurons/norm_smallim22.tif",
    "./data/raw_neurons/norm_smallim23.tif",
    "./data/raw_neurons/norm_smallim24.tif",
    "./data/raw_neurons/norm_smallim26.tif",
    "./data/raw_neurons/norm_smallim27.tif",
    "./data/raw_neurons/norm_smallim28.tif",
    "./data/raw_neurons/norm_smallim5.tif",
    "./data/raw_neurons/sw_norm_smalliim16.tif",
    "./data/raw_neurons/sw_norm_smallim11.tif",
    "./data/raw_neurons/sw_norm_smallim12.tif",
    "./data/raw_neurons/sw_norm_smallim13.tif",
    "./data/raw_neurons/sw_norm_smallim14.tif",
    "./data/raw_neurons/sw_norm_smallim15.tif",
    "./data/raw_neurons/sw_norm_smallim17.tif",
    "./data/raw_neurons/bad_Project_Series016.tif",
    "./data/raw_neurons/good_Project_Series008_z00.tif",
    "./data/raw_neurons/moded_good_Project_Series008_z00.tif",
    "./data/raw_neurons/moded_bad_Project_Series016.tif",
        ]  

# init blured and clear spheres
SPHERES_PATH = "./data/spheres_images/19.19.200_200nm_vital"
BEAD_SHAPE = [31, 63, 63]
Z_SCALE, X_SCALE, Y_SCALE = 0.2, 0.019, 0.019
BEAD_DIAMETER = 0.2    

# init constants for dataset
IMG_SPLIT_SHAPE = (30, 150, 150)
DATA_SHAPE = (36, 160, 160) 

# constants for dataset naming and saving
DATASET_NAME, DATASET_PATH = "inflated_new_vital", "./"
DESCRIPTION = "Inflated big dataset of vital microscopy. With ERs and calogen"

# ADVANCED PROPERTIES (select them by using 'average_bead_viewer.ipynb'
# Parameters for additional bluring clear sphere for more smooth intensity degradation
SIGMA = 4.0        # Gaussian sigma value
MIN_DIST = 0.150   # Maximum additional space, which be taken with sphere in each axis (in nm). Be attention: this value will be multiplied on 1.5 to OZ axis.

## Генерация данных

### 1. Подключение файлов и функций для запуска пайплайна генерации данных

In [ ]:
import typing as tp
import numpy as np

from pipeline_provider import Pipeline
from dataset_generator import DatasetGenerator
from image_processing import *
from utils import *
from spheres_processing import *


def noize_filtering(img : np.ndarray, gauss_blur_sigma : float = 3.5) -> np.ndarray:
    binarized_img = make_3d_binarization(img, gauss_blur_sigma)
    img[binarized_img == 0] = 0
    return img

def extention_bluring(img : np.ndarray, z_rad:int, x_rad:int, y_rad:int) -> np.ndarray:
    z_rad = max(z_rad, 1)
    x_rad = max(x_rad, 1)
    y_rad = max(y_rad, 1)
    blur_filter = np.zeros(shape=(2 * z_rad - 1, 2 * x_rad - 1, 2 * y_rad - 1))
    
    for z in range(2 * z_rad - 1):
        for x in range(2 * x_rad - 1):
            for y in range(2 * y_rad - 1):
                if ((z - z_rad + 1) / z_rad) ** 2 + ((x - x_rad + 1) / x_rad) ** 2 + ((y - y_rad + 1) / y_rad) ** 2 <= 1:
                    blur_filter[z, x, y] = 1
    blur_filter = blur_filter / np.sum(blur_filter)
    
    new_img = convolution(img, blur_filter)
    return new_img


def augmentate_imgs(img : np.ndarray, augmentation_cnt : int = 10) -> tp.List[np.ndarray]:
    augmentated_imgs = [img]
    each_piece_augmentate_piece = augmentation_cnt - 1
    
    min_crop_size = 1
    for j in range(each_piece_augmentate_piece):
        # crop along z intensities
        along_z_crop = np.random.randint(min_crop_size, img.shape[0])
        #min_crop_size = along_z_crop
        #new_img = crop_image(img, [along_z_crop, -1, -1])
        new_img = img
        
        # rotate img
        rotates_cnt = np.random.randint(0, 3) 
        if rotates_cnt != 0:
            new_img = np.rot90(new_img, rotates_cnt, (1, 2))
        
        # make extention blur
        rad_Z = np.random.randint(0, 7)
        rad_X = np.random.randint(0, 15)
        rad_Y = np.random.randint(0, 15)
        new_img = extention_bluring(new_img, rad_Z, rad_X, rad_Y)
        
        # blur image for smoothing edges
        sigma_Z = np.random.uniform(0, 1)
        sigma_X = np.random.uniform(0, 1)
        sigma_Y = np.random.uniform(0, 1)
        new_img = gaussian_filter(new_img, sigma=(sigma_Z, sigma_X, sigma_Y))
        
        # shift along axes
        shift_along_z = np.random.randint(-6, 6)
        shift_along_x = np.random.randint(-12, 12)
        shift_along_y = np.random.randint(-12, 12) 
        new_img = move_frame(new_img, [shift_along_z, shift_along_x, shift_along_y])
        
        # add noize for more different data
        noize_lvl = np.random.randint(0, 4)
        #new_img = add_poisson_noise(new_img, noize_lvl)
        
        # add median filter for variative noize smoothing 
        size_z = np.random.randint(1, 5)
        size_xy = np.random.randint(1, 15)
        new_img = median_blurring(new_img, size_tuple=(size_z, size_xy, size_xy))
        
        # add to augmentated stack
        augmentated_imgs.append(new_img)
        
    return augmentated_imgs


def generate_pair(img:np.ndarray, blured_bead:np.ndarray, clear_bead:np.ndarray) -> tp.Tuple[np.ndarray]:
    # generate data
    x_data = convolution(img, blured_bead)
    y_data = convolution(img, clear_bead)
        
    return (x_data, y_data)

def discretize_data_intensities(x_data:np.ndarray, y_data:np.ndarray, max_discr_val:int = 255) -> tp.Tuple[np.ndarray]:
    x_data = (np.round(x_data * max_discr_val)).astype("float32") / max_discr_val
    print(f"DISCR INFO: Blured sum: {np.sum(x_data)}, Clear sum: {np.sum(y_data)}, Blured max: {np.max(x_data)},  Clear max: {np.max(y_data)}")
    return (x_data, y_data)

def variate_intensity(x_data:np.ndarray, y_data:np.ndarray) -> tp.Tuple[np.ndarray]:
    max_value = uniform(0.25, 1.0)
    multipluer = max_value / np.amax(x_data)
    print(f"np.amax(x_data): {np.amax(x_data)}, max_value: {max_value}, multipluer: {multipluer}")
    x_data *= multipluer
    y_data *= multipluer
    
    print(f"VAR INFO: Blured sum: {np.sum(x_data)}, Clear sum: {np.sum(y_data)}, Blured max: {np.max(x_data)},  Clear max: {np.max(y_data)}")

    return (x_data, y_data)


def image_loading(path : str, image_max_value:int = 255) ->tp.List[np.ndarray]:
    # load image
    img = load_tiff(path)
    img = (img - np.min(img)).astype("float32") / (np.max(img) - np.min(img)) * image_max_value
    return [img]


def make_shifts_of_origin_image(img : np.ndarray, shifts_size : tp.Tuple[int], shifts_count : tp.Tuple[int]):
    new_img_list = []
    
    for y_shift in range(shifts_count[0]):
        for x_shift in range(shifts_count[1]):
            new_img = img[:, y_shift*shifts_size[0]:, x_shift*shifts_size[1]:]
            new_img_list.append(new_img)
            print(f"Appended new img. New img shape: {new_img.shape}, old shape: {img.shape}")
    
    print(f"Total size: {len(new_img_list)}")
    return new_img_list



def image_slicing(img : np.ndarray, image_split_shape : tp.Tuple[int], image_max_value:int = 255) ->tp.List[np.ndarray]:
    # binarize to find places where could be usefull structures and no background
    binarized_img = make_3d_binarization(img, gauss_blur_sigma = 3.5)
    binarized_img_pieces = np.array(split_image(binarized_img,
        min(image_split_shape[2], binarized_img.shape[2]), 
        min(image_split_shape[1], binarized_img.shape[1]), 
        min(image_split_shape[0], binarized_img.shape[0])))
    
    # select pieces which contains some info
    pieces_sums = np.array([np.sum(piece) for piece in binarized_img_pieces])
    print(f"splited binarised parts: {len(binarized_img_pieces)}")
    pieces_indxs = np.where(pieces_sums > 0)
    
    # ..and split it..
    img_pieces = np.array(split_image(img,
        min(image_split_shape[2], img.shape[2]), 
        min(image_split_shape[1], img.shape[1]), 
        min(image_split_shape[0], img.shape[0])))
    # ...and select some pieces with info
    img_pieces = img_pieces[pieces_indxs]
    img_pieces = [piece for piece in img_pieces]
    print(f"splited non-empty parts: {len(img_pieces)}")
    print(len(img_pieces), img_pieces[0].shape)
    return img_pieces

### 2. Генерация сфер

In [ ]:
import matplotlib.cm as cm
from plotting_utils import *

# Генерация сфер
gauss_blur_sigma = 1.5
blured_bead = generate_average_bead(SPHERES_PATH, BEAD_SHAPE, low_border=5, gauss_blur_sigma=gauss_blur_sigma)
clear_airy_bead = model_3d_airy_bead(BEAD_SHAPE, Z_SCALE, X_SCALE, Y_SCALE, BEAD_DIAMETER, zoomfactor=2.6)

# Дополнительное размытие
blured_clear_airy_bead = gaussian_blurring_sphere(clear_airy_bead, Z_SCALE, X_SCALE, Y_SCALE, SIGMA, additional_space_xy=MIN_DIST / 2, additional_space_z = MIN_DIST)

# normalize for using as filters
blured_bead = blured_bead / np.sum(blured_bead)
clear_bead = blured_clear_airy_bead / np.sum(blured_clear_airy_bead)

# ПРОВЕРКА: изображения-фильтры центрированы...
print(blured_bead.dtype, clear_bead.dtype)
print(np.unravel_index(clear_bead.argmax(), clear_bead.shape), clear_bead[np.unravel_index(clear_bead.argmax(), clear_bead.shape)])
print(clear_bead[clear_bead.shape[0] // 2, clear_bead.shape[1] // 2, clear_bead.shape[2] // 2])
print(np.unravel_index(blured_bead.argmax(), blured_bead.shape), blured_bead[np.unravel_index(blured_bead.argmax(), blured_bead.shape)])
# ...и сумма точек равна 1
print(np.sum(clear_bead))
print(np.sum(blured_bead))

# Отрисовка фильтров
plot_image_slices(blured_bead, cm.jet, X_SCALE, Z_SCALE, np.array(BEAD_SHAPE) // 2)
plot_image_slices(clear_bead, cm.jet, X_SCALE, Z_SCALE, np.array(BEAD_SHAPE) // 2)

In [ ]:
fig, axes = center_intensities_show(blured_clear_airy_bead)
fig.show()

### 3. Генерация данных

In [ ]:
# init dataset generator
dg = DatasetGenerator(DATASET_NAME, DATA_SHAPE, data_description=DESCRIPTION, path=DATASET_PATH)
dg.dump_data_info()

# Сохранение сфер как одной из формирующих частей датасета!
np.save(os.path.join(DATASET_PATH + DATASET_NAME, "blured_bead"), blured_bead)
np.save(os.path.join(DATASET_PATH + DATASET_NAME, "clear_bead"), clear_bead)

methods_list = [
    lambda path : image_loading(path),
    lambda img: make_shifts_of_origin_image(img, (75, 50), (2, 3)),
    lambda img: image_slicing(img, IMG_SPLIT_SHAPE),
    lambda img: [img] if np.sum(img) > 0 else [],                               # filter in pipeline
    lambda img: [img] if np.amax(img) > 15 else [],                               # filter in pipeline
    lambda img: augmentate_imgs(img, 3),
    lambda img: [img] if np.sum(img) > 0 else [],                               # filter in pipeline
    # NEW: WE FIRSTLY GENERATE AND AUGMENTATE SOME IMAGES, ONLY WHEN WE ARE INFLATE IT AND MAKE CONVOLUTION!
    lambda img: inflate_image(img, DATA_SHAPE),
    lambda img : generate_pair(img, blured_bead, clear_bead),
    lambda x_data, y_data : variate_intensity(x_data, y_data),
    lambda x_data, y_data : discretize_data_intensities(x_data, y_data),
    lambda x_data, y_data : dg.append(x_data, y_data) 
]

pipeline = Pipeline(methods_list)
for path in PATHS:
    pipeline(path) 
dg.dump_data_info()
dg.close()

In [ ]:
dg.dump_data_info()
dg.close()

In [ ]:
blured_bead = np.load("./dataset_w_paddings/blured_bead.npy")
clear_bead = np.load("./dataset_w_paddings/clear_bead.npy")
print(np.unravel_index(blured_bead.argmax(), blured_bead.shape), blured_bead[np.unravel_index(blured_bead.argmax(), blured_bead.shape)])
print(np.unravel_index(clear_bead.argmax(), clear_bead.shape), clear_bead[np.unravel_index(clear_bead.argmax(), clear_bead.shape)])


### Доп секция для визуализаций разных штук

In [ ]:
tmp_img_data = load_tiff("./data/raw_neurons/21.tiff")
print(tmp_img_data.shape)
save_image_slices(tmp_img_data, "./data/raw_neurons/21.png", cm.jet, 0.022, 0.1, np.array(tmp_img_data.shape) // 2)